In [2]:
import quaternion
import numpy as np

## Difference in ordering of quaternion multiplication



In [8]:
# Start with orientation pointing west
q = quaternion.Quaternion.axis_angle([0,0,1], np.pi/4)
print(q.euler_angles()*180/np.pi)
# Now rotate (small angle) 5 degrees in all axes
q = q*quaternion.Quaternion(1.0, .5*5*np.pi/180,.5*5*np.pi/180,.5*5*np.pi/180)
q = q*quaternion.Quaternion(1.0, .5*5*np.pi/180,.5*5*np.pi/180,.5*5*np.pi/180)
print(q.euler_angles()*180/np.pi)

[ 0.  0. 45.]
[11.01752596  9.10853741 56.4394748 ]


In [7]:
q = quaternion.Quaternion(1,0,0,0)
print(q.euler_angles()*180/np.pi)
# Now rotate (small angle) 5 degrees in all axes
q = quaternion.Quaternion(1.0, *(.5*5*np.pi/180*np.array([0,0,1.0])))*q
print(q.euler_angles()*180/np.pi)

[0. 0. 0.]
[0.         0.         5.00633166]


First off, let's do some sanity checking of how to use quaternions.

Let's represent a few body configurations and then see if the translations from inertial to body coordinate systems make sense.  The inertial coordinate system is x axis north and the z axis away from the center of the earth.  The body coordinate system is x in the direction of the vehicle travel (roll axis), y along the left wing (pitch axis) and z as yaw.

Let's represent the body as headed west.  The north vector in body coordinates should point along the right wing.

In [22]:
print("Headed west")
q = quaternion.Quaternion.axis_angle(np.array([0,0,1.0]), np.pi/2)
print("Euler angles (roll, pitch, yaw):", q.euler_angles()*180/np.pi)
# A vector pointing north ([1,0,0] in inertial coords), should be along right wing (negative y direction in body coordinates)
north_body = (q.inv()*quaternion.Quaternion.from_vec(np.array([1.0,0,0]))*q).as_ndarray()[1:]
print("North in body coordinates:", north_body)
print()

print("Pitch up 10 degrees")
q = quaternion.Quaternion.axis_angle(np.array([0,1.0,0]), -10*np.pi/180)
print("Euler angles (roll, pitch, yaw):", q.euler_angles()*180/np.pi)
# A vector pointing north ([1,0,0] in inertial coords), should be along right wing (negative y direction in body coordinates)
north_body = (q.inv()*quaternion.Quaternion.from_vec(np.array([1.0,0,0]))*q).as_ndarray()[1:]
print("North in body coordinates:", north_body)
print()

print("Headed west then banked left 20 degrees")
q = quaternion.Quaternion.axis_angle(np.array([0,0,1.0]), np.pi/2)
q = quaternion.Quaternion.axis_angle(np.array([1.0,0,0]), -20*np.pi/180)*q
print("Euler angles (roll, pitch, yaw):", q.euler_angles()*180/np.pi)
# A vector pointing north ([1,0,0] in inertial coords), should be along right wing (negative y direction in body coordinates)
north_body = (q.inv()*quaternion.Quaternion.from_vec(np.array([1.0,0,0]))*q).as_ndarray()[1:]
print("North in body coordinates:", north_body)
print()

print("Headed west, pitched up 5 degrees, rolled left 20 degrees")
q = quaternion.Quaternion.axis_angle(np.array([0,0,1.0]), np.pi/2)
q *= quaternion.Quaternion.axis_angle(np.array([0,1.0,0]), -5*np.pi/180)
q *= quaternion.Quaternion.axis_angle(np.array([1.0,0,0]), -20*np.pi/180)
print("Euler angles (roll, pitch, yaw):", q.euler_angles()*180/np.pi)
# A vector pointing north ([1,0,0] in inertial coords), should be along right wing (negative y direction in body coordinates)
north_body = (q.inv()*quaternion.Quaternion.from_vec(np.array([1.0,0,0]))*q).as_ndarray()[1:]
print("North in body coordinates:", north_body)
print()

Headed west
Euler angles (roll, pitch, yaw): [ 0.  0. 90.]
North in body coordinates: [ 0. -1.  0.]

Pitch up 10 degrees
Euler angles (roll, pitch, yaw): [  0. -10.   0.]
North in body coordinates: [ 0.98480775  0.         -0.17364818]

Headed west then banked left 20 degrees
Euler angles (roll, pitch, yaw): [ 0. 20. 90.]
North in body coordinates: [-6.9388939e-18 -1.0000000e+00  0.0000000e+00]

Headed west, pitched up 5 degrees, rolled left 20 degrees
Euler angles (roll, pitch, yaw): [-20.  -5.  90.]
North in body coordinates: [-1.31838984e-16 -9.39692621e-01 -3.42020143e-01]



# Create coordinate system if heading is known

If we know $\psi$, how do we create the resultant coordinate system?  $\psi$ can be determined from orientation quaternion from:
$$ \mbox{atan2}[2(q_0 q_3 + q_1 q_2),
                         1-2(q_2^2+q_3^2))]
                         $$

In [11]:
# Use psi from example above.
q = q.as_ndarray()
print("Extracted Psi:", np.arctan2(2*(q[0]*q[3] + q[1]*q[2]), 1-2*(q[2]**2 + q[3]**2))*180/np.pi)

# Checking that sign is preserved. Try rotation in other direction.
q = quaternion.Quaternion.axis_angle(np.array([0,0,1.0]), -np.pi/2)
q *= quaternion.Quaternion.axis_angle(np.array([0,1.0,0]), -5*np.pi/180)
q *= quaternion.Quaternion.axis_angle(np.array([1.0,0,0]), -20*np.pi/180)
print("Euler angles (roll, pitch, yaw):", q.euler_angles()*180/np.pi)
# A vector pointing north ([1,0,0] in inertial coords), should be along right wing (negative y direction in body coordinates)
north_body = (q.inv()*quaternion.Quaternion.from_vec(np.array([1.0,0,0]))*q).as_ndarray()[1:]
print("North in body coordinates:", north_body)
print()

Extracted Psi: 90.0
Euler angles (roll, pitch, yaw): [-20.  -5. -90.]
North in body coordinates: [1.87350135e-16 9.39692621e-01 3.42020143e-01]

